# Project: Protein stability prediction

In the project you will try to predict protein stability changes upon point mutations. 
We will use acuumulated data from experimental databases, i.e. the Megascale dataset. A current [paper](https://www.pnas.org/doi/10.1073/pnas.2314853121) has already preprocessed the dataset and created homology reduced data splits. We will reuse these. To do so, download the data folder from [here](https://polybox.ethz.ch/index.php/s/txvcb5jKy1A0TbY) and unzip it.  

The data includes measurements of changes in the Gibbs free enrgy ($\Delta \Delta G $). 
This will be the value that you will have to predict for a given protein with a point mutation. 
As input data you can use the protein sequence or a protein embedding retreived from ESM, a state of the art protein model.  

Here we will use the sequence as input. 
The model will predict the $\Delta \Delta G $ of point mutations in this sequence. To make training more efficient, the model should directly predict the values for all possible mutations at each position in the sequence. So the expected output is a sequence of $ L \ (sequence \ length) \ x \ 20 \ (number \ amino \ acids)$. This will be shown in detail later.

Below we provide you with a strcuture for the project that you can start with.  
Edit the cells to your liking and add more code to create your final model.

# Imports

In [1]:
import os 
import numpy as np
import pandas as pd
import scipy
import sklearn.metrics as skmetrics

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import lightning as L

import torchmetrics
from torchmetrics.regression import PearsonCorrCoef

/home/course/anaconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Dataloading

We are using the Megascale dataset. The train, validation and test sets are already predefined.
We provide code to load the data and helper functions to encode the sequences to numerical vectors (one-hot encoding). You can use this code as a starting point, adjust it, or use your own data loading. 

Each sequence will be treated as one batch (easier to deal with different leghts this way). The class below returns a dictionary containing the one-hot encoded sequence of dimension $Lx20$, the target sequence of dimension $Lx20$, containing the $\Delta \Delta G $ values, and a mask of the same dimension which indicates with a 1 if an experimental value is available for that position. Only compute your loss on the positions where an experimental value is available. So compute your loss similar to this example:

``` 
prediction = model(x)
loss = loss(prediction[mask==1],labels[mask==1])
```

In [28]:
aa_alphabet = 'ACDEFGHIKLMNPQRSTVWY' # amino acid alphabet
aa_to_int = {aa: i for i, aa in enumerate(aa_alphabet)} # mapping from amino acid to number

# function to one hot encode sequence
def one_hot_encode(sequence):
    # initialize a zero matrix of shape (len(sequence), len(amino_acids))
    one_hot = torch.zeros(len(sequence), len(aa_alphabet))
    for i, aa in enumerate(sequence):
        # set the column corresponding to the amino acid to 1
        one_hot[i].scatter_(0, torch.tensor([aa_to_int[aa]]), 1)
    return one_hot


# sequence data, comes already batched, so treat accordingly in dataloader (batch_size=1)
class SequenceData(Dataset):
    def __init__(self, csv_file, label_col="ddG_ML"):
        """
        Initializes the dataset. 
        input:
            csv_file: path to the relevant data file, eg. "/home/data/mega_train.csv"
        """
        self.df = pd.read_csv(csv_file, sep=",")
        self.label_col = label_col
        # only have mutation rows
        self.df = self.df[self.df.mut_type!="wt"]
        # process the mutation row
        self.df["mutation_pos"] = self.df["mut_type"].apply(lambda x: int(x[1:-1])-1) # make position start at zero
        self.df["mutation_to"] = self.df["mut_type"].apply(lambda x: aa_to_int[x[-1]]) # give numerical label to mutation

        # group by wild type
        self.df = self.df.groupby("WT_name").agg(list)
        # get wild type names
        self.wt_names = self.df.index.values
        # precompute one-hot encoding for faster training
        self.encoded_seqs = {}
        for wt_name in self.wt_names:
            # get the correct row
            mut_row = self.df.loc[wt_name]
            seq = mut_row["wt_seq"][0]
            self.encoded_seqs[wt_name] = one_hot_encode(seq)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # get the wild type name
        wt_name = self.wt_names[idx]
        # get the correct row
        mut_row = self.df.loc[wt_name]
        # get the wt sequence in one hot encoding
        sequence_encoding = self.encoded_seqs[wt_name]

        # create mask and target tensors
        mask = torch.zeros((len(sequence_encoding),20)) # will be 1 where we have a measurement
        target = torch.zeros((len(sequence_encoding),20)) # ddg values
        # all mutations from df
        positions = torch.tensor(mut_row["mutation_pos"])
        amino_acids = torch.tensor(mut_row["mutation_to"])
        # get the labels
        labels = torch.tensor(mut_row[self.label_col])

        for i in range(len(sequence_encoding)):
            mask[i,amino_acids[positions==i]] = 1 # one where we have data
            target[i,amino_acids[positions==i]] = labels[positions==i] # fill with ddG values
        
        # returns encoded sequence, mask and target sequence 
        return {"sequence": sequence_encoding[None,:,:].float()[0], "mask": mask, "labels": target}

In [29]:
# usage
dataset_train = SequenceData('project_data/mega_train.csv')
dataset_val= SequenceData('project_data/mega_val.csv')
dataset_test = SequenceData('project_data/mega_test.csv')

# use batch_size=1 bc we treat each sequence as one batch
dataloader_val = DataLoader(dataset_val, batch_size=1, shuffle=False)
dataloader_train = DataLoader(dataset_train, batch_size=1, shuffle=True)
dataloader_test = DataLoader(dataset_test, batch_size=1, shuffle=False)

## Data exploration

Explore the data and try to understand what we are working with. How is the data structured? How is it distributed? What do the values mean? How is the data represented and how else could it be represented? 
Your approach depends on your understanding of the data.

In [31]:

dataset_train[0]['sequence'].shape

torch.Size([63, 20])

In [32]:
dataloader_train

## Model architecture and training

Now it's your turn. Create a model trained on the given sequences.  
Be aware that this is not a classification task, but a regression task. You want to predict continuous numbers as close to the measured $\Delta \Delta G $ value as possible.
You will need to adjust your architecture and loss accordingly.

Train the model with the predefined dataloaders and try to improve it. 
Only test on the test set at the very end, when you have finished fine-tuning you model. 

In [33]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import torch
import torch.nn as nn

class LSTMPredictor(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers=1):
        super(LSTMPredictor, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)  # Output a single value (ddG)

    def forward(self, x):
        # x: (batch_size, seq_len, input_dim)
        lstm_out, _ = self.lstm(x)  # lstm_out: (batch_size, seq_len, hidden_dim)
        predictions = self.fc(lstm_out)  # predictions: (batch_size, seq_len, 1)
        return predictions

In [34]:
class MyLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MyLSTM,self).__init__()
        
        # input parameters
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        

        # define model layers (LSTM), pseudocode:
        self.LSTM = nn.LSTM(self.input_dim, self.hidden_dim, num_layers = 1, batch_first = True)
        # define full connected layer
        self.fc1 = nn.Linear(self.hidden_dim, self.output_dim)

    def forward(self,inp):
        inp1 = inp.to(device)
        c0 = torch.zeros(1, inp.size(0), self.hidden_dim).double().to(device)
        h0 = torch.zeros(1, inp.size(0), self.hidden_dim).double().to(device)
        #print("Input shape:", inp.shape)
        #print('inp:', inp1)
        #print('c0', c0)
        #print('h0', h0)
        #get output for all time steps
        out, (hn, cn) = self.LSTM(inp1, (h0, c0))

        #apply fc to all steps
        output= self.fc1(out)
     #   print("output shape:", output.shape)
        return output

In [39]:
####lightning module to train the sequence model
class SequenceModelLightning(L.LightningModule):
    def __init__(self, input_dim, hidden_dim, output_dim, learning_rate=0.1):
        super().__init__()
        self.model = MyLSTM(input_dim, hidden_dim, output_dim).double()
        self.learning_rate = learning_rate
        # define loss function here, pseudocode:
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        input_tensor = batch['sequence'].double()
        target_tensor = batch['labels'].double()
        
        output = self.model(input_tensor)
        loss = self.loss(output, target_tensor)
     #   loss = self.loss(output.view(-1, output.shape[2]),target_tensor.view(-1).long())
        self.log("train_loss", loss, prog_bar=True)
     #   print('training good')
        return loss
    
    def validation_step(self, batch, batch_idx):
        input_tensor = batch['sequence'].double()
        target_tensor = batch['labels'].double()
     #   print("target shape:", target_tensor.shape)
     #   print('batchsize', input_tensor[0].shape)
        output = self.model(input_tensor)
     #   print("output shape:", output.shape)
        loss = self.loss(output, target_tensor)
     #   loss = self.loss(output.view(-1, output.shape[2]),target_tensor[0].view(-1).long())
        self.log("val_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
     #   print('validation good')
        return loss
    
    def configure_optimizers(self):
        # define optimizer here
        optimizer = torch.optim.Adam(self.model.parameters(), lr=0.01)
        return optimizer

In [40]:
lit_model = SequenceModelLightning(input_dim = 20,
                                  hidden_dim = 10,
                                  output_dim = 20,
                                  learning_rate = .001)

# define the trainer
trainer = L.Trainer(devices = 1, 
                    max_epochs = 10)

# learn the weights of the model
trainer.fit(lit_model, dataloader_train, dataloader_val)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params | Mode 
---------------------------------------------------
0 | model | MyLSTM           | 1.5 K  | train
1 | loss  | CrossEntropyLoss | 0      | train
---------------------------------------------------
1.5 K     Trainable params
0         Non-trainable params
1.5 K     Total params
0.006     Total estimated model params size (MB)


Epoch 0: 100%|█| 239/239 [00:01<00:00, 189.84it/s, v_num=28, train_loss=-1.15e+3
idation: |                                             | 0/? [00:00<?, ?it/s]
idation DataLoader 0: 100%|█████████████████| 31/31 [00:00<00:00, 251.81it/s]
Epoch 1: 100%|█| 239/239 [00:01<00:00, 191.65it/s, v_num=28, train_loss=-2.74e+3
idation: |                                             | 0/? [00:00<?, ?it/s]
idation DataLoader 0: 100%|█████████████████| 31/31 [00:00<00:00, 243.36it/s]
Epoch 2: 100%|█| 239/239 [00:01<00:00, 193.51it/s, v_num=28, train_loss=-2.47e+3
idation: |                                             | 0/? [00:00<?, ?it/s]
idation DataLoader 0: 100%|█████████████████| 31/31 [00:00<00:00, 267.02it/s]
Epoch 3: 100%|█| 239/239 [00:01<00:00, 195.00it/s, v_num=28, train_loss=-9.28e+3
idation: |                                             | 0/? [00:00<?, ?it/s]
idation DataLoader 0: 100%|█████████████████| 31/31 [00:00<00:00, 256.69it/s]
Epoch 4: 100%|█| 239/239 [00:01<00:00, 190.24it/s, v

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|█| 239/239 [00:01<00:00, 173.82it/s, v_num=28, train_loss=-2.9e+4,


# Hyperparameters
input_dim = 20  # One-hot encoding size
hidden_dim = 64  # LSTM hidden dimension
num_layers = 1  # Number of LSTM layers
learning_rate = 0.001
num_epochs = 10


model.eval()
with torch.no_grad():
    total_loss = 0
    for batch in dataloader_val:
        sequence = batch["sequence"]
        mask = batch["mask"]
        labels = batch["labels"]

        predictions = model(sequence)
        loss = masked_mse_loss(predictions, labels, mask)
        total_loss += loss.item()

    print(f"Validation Loss: {total_loss / len(dataloader_val):.4f}")

## Validation and visualization

To get a good feeling of how the model is performing and to compare with literature, compute the Pearson and Spearman correlations.
You can also plot the predictions in a scatterplot. We have added some code for that. 

In [43]:
preds =[]
all_y = []

for batch in dataloader_val:
    # read from batch
    x = batch["sequence"].to(device)
    mask = batch["mask"].to(device)
    target = batch["labels"].to(device)
    model = lit_model.to(device)
    ## adjust to work with your model
    # predict
    prediction = model(x)
    preds.append(prediction[mask==1].flatten().detach().numpy()) # flatten to create one dimensional vector from 2D sequence
    all_y.append(target[mask==1].flatten().detach().numpy()) # flatten to create one dimensional vector from 2D sequence

# concatenate and plot
preds= np.concatenate(preds)
all_y = np.concatenate(all_y)

sns.regplot(x=preds,y=all_y)
plt.xlabel("Predicted ddG")
plt.ylabel("Measured ddG")

# get RMSE, Pearson and Spearman correlation 
print("RMSE:", skmetrics.mean_squared_error(all_y, preds, squared="False"))
print("Pearson r:", scipy.stats.pearsonr(preds, all_y))
print("Spearman r:", scipy.stats.spearmanr(preds, all_y))

RuntimeError: Input and hidden tensors are not the same dtype, found input tensor with Float and hidden tensor with Double

## Interpretation

Try to analyse and interpret your model and/or predictions in the context of the biological question.